<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/%EA%B0%9C%EB%B0%9C%EC%9E%90%EB%A5%BC%EC%9C%84%ED%95%9C%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EB%94%A5%EB%9F%AC%EB%8B%9D/part1.%20%EB%AA%A8%EB%8D%B8%EA%B5%AC%EC%B6%95/5_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC%20/5_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 자연어 처리 소개

#### 토큰화

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]

tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}


Tokenizer 객체를 만들 때 토큰화할 수 있는 단어 개수를 지정한다. 이값은 말뭉치에서 추출할 수 있는 최대 토큰 개수이다.

fit_on_texts를 호출해 토큰화된 단어 인덱스를 만든다. 이 객체의 word_index 속성을 출력하면 말뭉치 안에 있는 단어와 인덱스의 키/값 쌍이 출력된다.

이 말뭉치는 매우 유연하다. today?는 today로 필터링

이런 동작은 Tokenizer 클래스의 filter 매개변수로 저잘한다. 이 매개변수의 기본값은 작은 따옴표를 제외한 모든 구두점을 제거한다.

따라서 앞의 인코딩에서 Today is a sunny day는 [1,2,3,4,5]가 되고, Is it sunny today는 [2,7,4,1]이 된다.

#### 문장을 시퀀스로 바꾸기

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[1, 2, 3, 4, 5], [1, 2, 3, 6, 5], [2, 7, 4, 1]]


자연어처리의 경우 모든 문맥에서 사용 가능한 모든 단어가 포함되어 있지 않다. 신경망이 이제까지 본 적 없는 단어가 포함된 새로운 텍스트를 만나게 되면 어떤 일이 일어날까?

새로운 단어의 문맥을 모르기 떄문에 결과적으로 예측에 부정적인 영향을 미친다.


#### OOV 토큰 사용하기


이런 상황을 처리하는 한 가지 방법은 OOV 토큰이다. 이 토큰은 신경망이 이전에 본 적 없는 단어가 포함된 텍스트의 문맥을 이해하는 데 도움이 된다

In [ ]:
test_data = [
    'Today is a snowy day',
    'Will it be rainy tommorow?'
    ]

이 입력을 (훈련데이터인) 기존 텍스트 말뭉치에 추가하는 것이 아니라 사전에 훈련된 신경망이 이 텍스트를 만났다고 생각해보자.

기존에 훈련한 Tokenizer 객체로 이 데이터를 토큰화 하면 다음과 같다.

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_seq)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}
[[1, 2, 3, 5], [7, 6]]


새로운 문장으로 만든 시퀀스에서 토큰을 단어로 바꾸면 'Today is a day 와 it rainy가 된다.

대부분의 문맥과 의미를 잃어버렸다. 이때 OOV 토큰이 필요하다.

OOV 토큰은 Tokenizer 객체를 만들 때 지정할 수 있으며 oov_token 매개변수로 설정한다.

어떤 문자로도 지정할 수 있지만 현재 말뭉치에 등장하지 않는 문자열이어야 한다.

In [ ]:
tokenizer = Tokenizer(num_words=100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

test_seq = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_seq)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


토큰 리스트에 새로운 항목인 < OOV > 가 추가되었고, 시퀀스의 길이도 원본 텍스트 길이 그대로 유지되었다.

출력 결과를 문장으로 복원하면 'Today is a < OOV > day'와

'< OOV > it < OOV > rainy < OOV >'가 된다.

첫 번째 문장은 원래 문장의 의미와 매우가깝다. 두번쨰 문장은 대부분의 단어가 훈련데이터에 없기 떄문에 문맥을 많이 잃었지만 확실히 더 나은 토큰화 방식이다.

##### TextVectorization 층 사용하기 (참고)

In [ ]:
tv = keras.layers.TextVectorization()
tv.adapt(sentences) # 어휘사전 만들기

In [ ]:
tv.get_vocabulary() # 학습된 어휘사전 확인

['', '[UNK]', 'today', 'is', 'sunny', 'day', 'a', 'rainy', 'it']

today는 인덱스 2, is 는 인덱스 3이 되는 식이다.

이 클래스는 < OOV > 토큰은 [ 'UNK' ] 로 표시하며 단어 인덱스는 1에 해당한다.

인덱스 0에 해당하는 토큰은 비워져있다. 일반적으로 인덱스 0은 패딩을 위해 사용한다. 이는 잠시 후에 알아본다.

새로운 문장을 시퀀스로 바꾸려면 tv 객체를 함수처럼 호출하면 된다. 반환된 값은 텐서이므로 numpy() 메서드를 사용해 넘파이 배열로 바꿀 수 있다.

In [ ]:
test_seq = tv(test_data)
test_seq.numpy()

array([[2, 3, 6, 1, 5],
       [1, 8, 1, 7, 1]])

#### 패딩 이해하기

신경망을 훈련할 때 일반적으로 모든 데이터 크기는 동일해야 한다.

텍스트에서는 단어를 토큰화하고 문장을 토큰의 시퀀스로 바꾸면 시퀀스마다 길이가 다르다. 이때 동일한 길이로 맞추기 위해 **패딩** 을 사용한다

In [ ]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?',
    'I really enjoyed walking in the snow today'
]

tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[2, 3, 4, 5, 6], [2, 3, 4, 7, 6], [3, 8, 5, 2], [9, 10, 11, 12, 13, 14, 15, 2]]


길이가 서로 다른 것을 볼 수 있다.

동일한 길이로 만들려면 **pad_sequences** 함수를 사용할 수 있다. 우선 이 함수를 임포트한다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

이 함수 사용볍은 쉽다. pad_sequences 함수를 호출해 패딩되지 않은 문장을 패딩된 문장으로 바꾼다.

In [ ]:
padded = pad_sequences(sequences)

print(padded)

[[ 0  0  0  2  3  4  5  6]
 [ 0  0  0  2  3  4  7  6]
 [ 0  0  0  0  3  8  5  2]
 [ 9 10 11 12 13 14 15  2]]


0으로 패딩. 토큰 리스트의 인덱스가 1부터 시작하는 이유이다.

pad_sequences의 기본값은 짧은 문장을 긴 문장의 길이에 맞추기 위해 시작부분을 0으로 패딩하는 prepadding이다.

padding 매개변수에서 이 값을 바꿀 수 있다. 예를 들어 끝에 0을 패딩하려면 다음과 같다.

In [ ]:
padded = pad_sequences(sequences, padding='post')
print(padded)

[[ 2  3  4  5  6  0  0  0]
 [ 2  3  4  7  6  0  0  0]
 [ 3  8  5  2  0  0  0  0]
 [ 9 10 11 12 13 14 15  2]]


끝에 0 추가된 걸 볼 수 있다.

기본적으로 모든 문장은 가장 긴 문장의 길이에 맞춰 패딩한다. 이렇게 하면 잃는 데이터는 없는 대신 패딩이 많아진다.

아주긴 문장 하나에 맞추기 위해 패딩이 너무 많이 추가되는 걸 원치 않는다면 **maxlen** 매개변수에 최대길이를 지정하면 된다.

In [ ]:
padded = pad_sequences(sequences, padding='post', maxlen=6)
print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [11 12 13 14 15  2]]


In [ ]:
# 시작부분 대신 끝 부분을 자르려면 truncating 매개변수 사용
padded = pad_sequences(sequences, padding='post', maxlen=6, truncating='post')
print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [ 9 10 11 12 13 14]]


### 불용어 제거와 텍스트 정제

실전 데이터셋에는 원치 않는 텍스트가 들어있는 경우가 종종 있다.

'the' , 'and, 'but' 과 같이 너무 자주 등장해 특별한 의미가 없는 **불용어** 를 제거할 수 있다.

그리고 텍스트에 들어있는 HTML 태그를 제거하고 싶을 수 있다.

무례한 단어, 구두점, 고유명사 등등의 불용어


   

   .

말뭉치마다 각기 작업이 다르지만 프로그래밍적으로 텍스트를 정제하는 방식에는 세가지 주요 단계가 있다.

1. 먼저 HTML 태그를 제거한다. 다행히 BeautifulSoup 라이브러리를 사용하면 간단하게 처리 가능.

In [ ]:
# from bs4 import BeautifulSoup

# soup = BeautifulSoup(sentence)
# sentence = soup.get_text()

불용어를 제거하는 일반적인 방법은 불용어 리스트를 준비하고 문장에서 이를 제거하여 전처리하는 방식이다. 다음 예와 같다.

In [ ]:
# stopwords = ['a', 'about', ...'yours'..]

2. 다음은 문장을 순회하며 불용어 제거하는 코드이다.

In [ ]:
# words = sentence.split()
# filtered_sentence = ''
# for word in words:
#   if word not in stopwords:
#     filtered_sentence = filtered_sentence + word + " "
# sentences.append(filtered_sentence)

3. 또 다른 작업은 불용어 삭제를 훼방할 수 있는 구두점을 제거하는 일이다.

앞의 코드는 공백으로 둘러싸인 불용어만 찾기 때문에 마침표나 쉼표 앞에 오는 불용어 찾지 못한다.

파이썬 string 패키지에서 제공하는 **translate** 함수를 사용하면 이런 문제를 쉽게 해결할 수 있다.

또한 **string.punctuation** 으로 일반적인 구두점 목록을 제공하기 때문에 다음 코드로 단어에서 구두점 제거 가능하다.

In [ ]:
# import string
# table = str.maketrans('', '', string.punctuation) # 원래문자, 대입할 문자, 제거할 문자
# words = sentence.split()
# filtered_sentence = ''

# for word in words:
#   word = word.translate(table)
#   if word not in stopwords:
#     filtered_sentence = filtered_sentence + word + " "
# sentences.append(filtered_sentence)

문장을 나누어 'it;' 과 같은 단어가 나왔다면 'it'으로 변환된 다음 뷸용어 제거.

하지만 이런 작업을 할 때 불용어 목록을 업데이트해야 할 수도 있다. 이 목록에는 보통 'you ll' 같은 축약형이 포함된다.

translate 함수는 'you ll' 을 'youll'로 바꾸므로 이를 제거하려면 불용어 목록에 이를 포함시켜야 한다.

여기까지의 세 단계를 따르면 훨씬 정제된 텍스트를 준비할 수 있다. 물론 데이터셋에 있는 고유한 특징은 별도의 처리 필요

### 실제 데이터 다루기

텐서플로 데이터셋에서 이미 많은 작업이 전처리된 IMDb 데이터셋을 다뤄보겠다.

 그다음 JSON 기반의 데이터셋과 CSV에 저장된 감성데이터셋을 처리하는 작업을 해보겠다.

#### 텐서플로 대이터셋에서 텍스트 가져오기

영화리뷰 50,000 개로 구성된 IMDB의 데이터셋 imdb_reviews를 살펴보겠다. 이 데이터는 긍정과 부정으로 레이블되어 있다.

In [ ]:
import tensorflow_datasets as tfds

imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split='train'))
for item in train_data:
  imdb_sentences.append(str(item['text']))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteF6B7US/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteF6B7US/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteF6B7US/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


문장이 준비되면 이전처럼 Tokenizer 객체를 만들고 훈련해 토큰의 시퀀스로 변환한다

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(imdb_sentences)
seq = tokenizer.texts_to_sequences(imdb_sentences)

단어 인덱스를 확인하기 위해 출력

In [ ]:
print(tokenizer.word_index)

**HTML 태그 제거, 구두점 삭제, 불용어 삭제해서 정제**

대시(ex. annoying-conclusion) 나 슬래시 (him/her)을 사용해 단어를 연결하는 경우가 리뷰엔 많다.

그래서 더욱 좋은 결과를 얻기 위해서 앞 뒤에 공백으로 구분을 해주는 작업을 추가해주었다.

In [ ]:
from bs4 import BeautifulSoup
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation) # 구두점 제거하기 위한 테이블

imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split="train")) # 데이터 불러옴
for item in train_data:
    sentence = str(item['text'].decode('UTF-8').lower()) # 훈련데이터 문장으로 변환
    sentence = sentence.replace(",", " , ")  # 리뷰에서 자주 쓰이는 문자도 구분하도록 공백 추가
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table) # 구두점 제거
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    imdb_sentences.append(filtered_sentence)


tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=25000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)
# 출력이 길어 노트북에서 제외함
print(tokenizer.word_index)

In [ ]:
sentences = [
    'Today is sunny day',
    'Today is rainy day',
    'Is it sunny today?'
]
seq = tokenizer.texts_to_sequences(sentences)
print(seq)

[[516, 5229, 147], [516, 6489, 147], [5229, 516]]


이를 디코딩하면 불용어가 빠지고 'today sunny day', 'today rainy day', 'sunny today' 같은 문장을 얻게 된다.

이를 코드로 작성하려면 키/값을 반대로 바꾼 새로운 딕셔너리를 만들어 숫자를 단어로 바꾸어야 한다.

In [ ]:
reverse_word_index = dict(
    [(value, key) for (key, value) in tokenizer.word_index.items()]
)

decoded_review = ' '.join([reverse_word_index.get(i, '?') for i in seq[0]])

print(decoded_review)

today sunny day


#### IMDb 부분 단어 데이터셋 사용하기

텐서플로 데이터셋은 부분 단어를 사용해 전처리된 IMDb 데이터셋 몇 가지도 제공한다.

이런 데이터셋에서는 문장을 단어로 나눌 필요가 없다. 미리 부분 단어로 나뉘어져 있음.

부분 단어는 말뭉치를 문자로 나누는 것(토큰 수는 적지만 의미가 부족하다) 과 단어로 나누는 것(토큰 수가 많지만 의미는 풍부하다) 사이에 좋은 절충안이다.

언어를 위한 분류기를 훈련하는 데 매우 효율적인 방식이다. 이런 데이터셋은 말뭉치를 나누고 인코딩하기 위한 인코더와 디코더도 포함한다.

**다음처럼 'imdb_reviews/subwords8k'나 'imdb_reviews/subword32k'를 전달해 tfds.load 호출**

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised=True,
    with_info=True
    )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteWUI8WS/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteWUI8WS/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteWUI8WS/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


다음처럼 info 객체를 통해 인코더를 참조할 수 있다. vocab_size를 확인해보겠다.

In [ ]:
encoder = info.features['text'].encoder
print('어휘 사전 크기: {}'.format(encoder.vocab_size))

어휘 사전 크기: 8185


In [ ]:
print(encoder.subwords) # 부분단어 목록 확인

출력에는 불용어, 구두점, 문법요소, < br > 같은 HTML 태그가 모두 들어있다.

공백은 밑줄 문자(_)로 표현된다. 따라서 첫 번쨰 토큰은 'the' 이다.

인코더의 encode 메서드를 사용해 문자열을 인코딩할 수 있다.

In [ ]:
sample_string = 'Today is a sunny day'

encoded_string = encoder.encode(sample_string)
print('인코딩된 문자열: {}'.format(encoded_string))

인코딩된 문자열: [6427, 4869, 9, 4, 2365, 1361, 606]


5개의 단어가 7개의 토큰으로 인코딩되었다.

토큰을 확인하려면 인코더의 subwords 속성을 사용한다. 'Today'에 있는 'Tod' 가 6427로 인코딩 되었다.

subwords 배열은 인덱스가 0부터 시작하므로 이 토큰은 6,426 번째 항목이다.

In [ ]:
print(encoder.subwords[6426])

Tod


토큰을 디코딩하려면 인코더의 decode 메서드 사용

In [ ]:
encoded_string = encoder.encode(sample_string)

original_string = encoder.decode(encoded_string)
test_string = encoder.decode([6427, 4869, 9, 4, 2365, 1361, 606])

print(original_string+ '/' + test_string)

Today is a sunny day/Today is a sunny day


#### CSV 파일에서 텍스트 읽기

이번에 사용할 데이터는 한 줄에 두개의 값을 담고있다.

첫 번째는 값이 긍정인지 부정인지(0 or 1)이고, 두번쨰는 텍스트 문자열이다.

데이터를 먼저 전처리하겠다.

In [ ]:
import csv
sentences=[]
labels=[]
with open('binary-emotion.csv', encoding='UTF-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    for row in reader:
        labels.append(int(row[0]))
        sentence = row[1].lower()
        sentence = sentence.replace(",", " , ")
        sentence = sentence.replace(".", " . ")
        sentence = sentence.replace("-", " - ")
        sentence = sentence.replace("/", " / ")
        soup = BeautifulSoup(sentence)
        sentence = soup.get_text()
        words = sentence.split()
        filtered_sentence = ""
        for word in words:
            word = word.translate(table)
            if word not in stopwords:
                filtered_sentence = filtered_sentence + word + " "
        sentences.append(filtered_sentence)

<ipython-input-68-dc289d6ccc73>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


이렇게 하면 결과로 35,327 개의 문장 리스트가 생성된다.

#### 훈련세트와 테스트 세트 만들기

텍스트 말뭉치를 문장 리스트로 읽었으므로 모델 훈련을 위해 이를 훈련세트와 테스트 세트로 나누어야 한다.

예를 들어 28,000개의 문장을 훈련에 사용하고 나머지를 테스트로 사용한다.

In [ ]:
training_size = 28000

training_sentences = sentences[0:training_size]
test_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
test_labels = labels[training_size:]

훈련 세트가 준비되었으므로 단어 인덱스를 만들어야 한다.

다음 코드는 **Tokenizer** 클래스를 사용해 단어 20,000 개로 구성된 어휘 사전을 만든다.

최대 문장 길이를 10개 단어로 하고, 이보다 문장이 길면 끝부분을 자르고, 짧으면 끝에 패딩을 추가한다.

'< OOV >' 토큰 사용을 한다

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 20_000
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length,
                                padding=padding_type, truncating = trunc_type)

training_sequences 와 training_padded 를 출력해 결과를 확인할 수 있다.

예를 들어 훈련 시퀀스의 첫 번쨰 항목을 출력해 최대 길이 10으로 어떻게 패딩되었는지 볼 수 있다.

In [ ]:
print(training_sequences)
print(training_padded)

단어 인덱스도 출력할 수 있다.

In [ ]:
print(tokenizer.word_index)

결과에 'like', 'dont' 같이 제거할 만한 불용어 단어가 많다. 단어 인덱스 조사는 언제나 도움이 된다.

#### JSON 파일에서 텍스트 읽기
자주 사용하는 또 다른 텍스트 파일 포맷은 JSON이다.

특히 웹 애플리케이션에서 데이터 교환을 위해 많이 사용되는 공개 표준 파일 포맷이다. 사람이 알아보기 쉽고, 이름/값 쌍을 사용하도록 고안되었다.

덕분에 레이블된 텍스트와 잘 맞다

**JSON 문법** 은 매우 간단하다. 중괄호 안에 이름/값 쌍으로 구성된 객체가 포함되며 콤마로 구분된다

In [ ]:
{"firstName":"Laurence",
 "lastName":"Moroney"}

{'firstName': 'Laurence', 'lastName': 'Moroney'}

In [ ]:
{
  {"firstName":"Laurence",
 "lastName":"Moroney"},
  {"firstName":"Sharon",
 "lastName":"Agathon"}
}

객체 안에 배열이 포함될 수도 있다.

In [ ]:
{
  {"firstName":"Laurence",
  "lastName":"Moroney",
  "emails" : ['sadasd@asd.com', 'asd21@azx.com']
 },
  {"firstName":"Sharon",
 "lastName":"Agathon"},
  "emails" : ['sadasd@asd.com', 'asd21@azx.com']
}

sarcasm.json 파일을 불러올 건데 구조는 다음과 같다

In [ ]:
{'is_sarcastic': 1 or 0
 'headline':string containing headline,
 'articel_link':string containing link
 }

#### JSON 파일 읽기

In [ ]:
import json
with open("sarcasm.json", 'r') as f:
  datastore = json.load(f)
  for item in datastore:
    sentence = item['headline'].lower()
    label = item['is_sarcastic']
    link = item['article_link']

**전처리 코드**

In [ ]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []
urls = []
for item in datastore:
    sentence = item['headline'].lower()
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    sentences.append(filtered_sentence)
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

<ipython-input-87-840f976f7c05>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


**훈련세트, 테스트 세트 나누기**

In [ ]:
training_size = 23_000

training_sentences = sentences[0:training_size]
test_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
test_labels = labels[training_size:]


**토큰화하고 훈련에 맞게 준비하는 과정도 이전과 동일**

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 20_000
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length,
                                padding=padding_type, truncating = trunc_type)

print(word_index)